# SG Job Analysis Dashboard

This Jupyter notebook focuses on analyzing job data from Singapore (SG) to build an interactive dashboard. It covers data exploration, cleaning, visualization, and key insights into the job market trends.

In [1]:
# import data from google colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = 'content/drive/MyDrive/NTU/SGJobData.csv'
except ImportError:
    print("Running locally, skipping Google Drive mount.")
    data_path = '../data/SGJobData.csv.xz'  # Local path to the dataset


Running locally, skipping Google Drive mount.


In [2]:
# load data into pandas dataframe and print first 5 rows
import pandas as pd
df = pd.read_csv(data_path)

print("Total data rows:", len(df))
print("First 5 rows of the dataset:")
df.head()


Total data rows: 1048585
First 5 rows of the dataset:


,categories,employmentTypes,metadata_expiryDate,metadata_isPostedOnBehalf,metadata_jobPostId,metadata_newPostingDate,metadata_originalPostingDate,metadata_repostCount,metadata_totalNumberJobApplication,metadata_totalNumberOfView,...,occupationId,positionLevels,postedCompany_name,salary_maximum,salary_minimum,salary_type,status_id,status_jobStatus,title,average_salary
0,"[{""id"":13,""category"":""Environment / Health""},{...",Permanent,2023-05-08,False,MCF-2023-0252866,2023-04-08,2023-03-30,2,5,151,...,NaN,Executive,WORKSTONE PTE. LTD.,2800,2000,Monthly,0,Closed,Food Technologist - Clementi | Entry Level | U...,2400.0
1,"[{""id"":21,""category"":""Information Technology""}]",Permanent,2023-05-08,False,MCF-2023-0273977,2023-04-08,2023-04-08,0,0,55,...,NaN,Executive,TRUST RECRUIT PTE. LTD.,5500,4000,Monthly,0,Closed,"Software Engineer (Fab Support) (Java, CIM, Up...",4750.0
2,"[{""id"":33,""category"":""Repair and Maintenance""}]",Full Time,2023-04-22,False,MCF-2023-0273994,2023-04-08,2023-04-08,0,7,99,...,NaN,Senior Executive,PU TIEN SERVICES PTE. LTD.,4600,3800,Monthly,0,Closed,Senior Technician,4200.0
3,"[{""id"":21,""category"":""Information Technology""}]",Permanent,2023-05-08,False,MCF-2023-0273991,2023-04-08,2023-04-08,0,6,113,...,NaN,Senior Executive,TRUST RECRUIT PTE. LTD.,10000,5000,Monthly,0,Closed,"Senior .NET Developer (.NET Core, MVC, MVVC, S...",7500.0
4,"[{""id"":2,""category"":""Admin / Secretarial""}]",Full Time,2023-05-08,False,MCF-2023-0273976,2023-04-08,2023-04-08,0,3,99,...,NaN,Non-executive,EATZ CATERING SERVICES PTE. LTD.,3400,2400,Monthly,0,Closed,Sales / Admin Cordinator,2900.0


In [3]:
# Extract 'category' values from JSON strings in the 'categories' column
import json

# Function to parse the JSON string and extract the 'category' value
def extract_category_names(categories_str):
    if pd.isna(categories_str) or categories_str == '[]':
        return None
    try:
        # Ensure the string is treated as valid JSON
        # Some entries might have single quotes or other issues, attempting to fix common ones
        categories_str = categories_str.replace("\\'", "'").replace("'", '"')
        categories_list = json.loads(categories_str)
        category_names = [item['category'] for item in categories_list if 'category' in item]
        return ', '.join(category_names)
    except json.JSONDecodeError:
        print(f"Warning: Could not decode JSON for entry: {categories_str[:100]}...")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e} for entry: {categories_str[:100]}...")
        return None

In [4]:
# Apply the function to create the new 'category_name' column
df['category_name'] = df['categories'].apply(extract_category_names)

print("New 'category_name' column created.")

# Ensure 'category_name' is not null and count occurrences
category_counts = df['category_name'].dropna().value_counts()

# Get the top 20 categories
top_20_categories = category_counts.head(20)

print("Total job categories extracted:", df['category_name'].nunique())
print("Top 20 Job Categories by Number of Records:")

# Convert the Series to a DataFrame for better table display
print(top_20_categories.reset_index().rename(columns={'index': 'Category Name', 'count': 'Number of Records'}))

# Get the bottom 20 categories
bottom_20_categories = category_counts.tail(20)

# Convert the Series to a DataFrame for better table display
print("Bottom 20 Job Categories by Number of Records:")
print(bottom_20_categories.reset_index().rename(columns={'index': 'Category Name', 'count': 'Number of Records'}))


New 'category_name' column created.
Total job categories extracted: 21125
Top 20 Job Categories by Number of Records:
                             category_name  Number of Records
0                   Information Technology              92869
1                              Engineering              49494
2                                      F&B              48461
3         Accounting / Auditing / Taxation              44720
4                Building and Construction              44374
5                      Admin / Secretarial              40187
6                           Sales / Retail              31339
7                 Logistics / Supply Chain              31318
8                   Education and Training              29987
9                      Banking and Finance              26661
10                                  Others              24648
11             Healthcare / Pharmaceutical              23368
12                         Human Resources              20190
13            

In [5]:
# display column names
print(df.columns)

Index(['categories', 'employmentTypes', 'metadata_expiryDate',
       'metadata_isPostedOnBehalf', 'metadata_jobPostId',
       'metadata_newPostingDate', 'metadata_originalPostingDate',
       'metadata_repostCount', 'metadata_totalNumberJobApplication',
       'metadata_totalNumberOfView', 'minimumYearsExperience',
       'numberOfVacancies', 'occupationId', 'positionLevels',
       'postedCompany_name', 'salary_maximum', 'salary_minimum', 'salary_type',
       'status_id', 'status_jobStatus', 'title', 'average_salary',
       'category_name'],
      dtype='object')


In [6]:
# Check columns with missing values
missing_values_count = df.isnull().sum()

# Filter to show only columns with missing values
columns_with_missing_values = missing_values_count[missing_values_count > 0]

if not columns_with_missing_values.empty:
    print("Columns with missing values and their counts:")
    for column, count in columns_with_missing_values.items():
        print(f"- {column}: {count} missing values")
else:
    print("No columns with missing values found.")

Columns with missing values and their counts:
- categories: 3988 missing values
- employmentTypes: 3988 missing values
- metadata_expiryDate: 3988 missing values
- metadata_jobPostId: 3988 missing values
- metadata_newPostingDate: 3988 missing values
- metadata_originalPostingDate: 3988 missing values
- occupationId: 1048585 missing values
- positionLevels: 3988 missing values
- postedCompany_name: 3988 missing values
- salary_type: 3988 missing values
- status_jobStatus: 3988 missing values
- title: 3988 missing values
- category_name: 3988 missing values


In [7]:
# Remove rows where 'categories' is null or empty
initial_rows = len(df)
df = df.dropna(subset=['categories'])
final_rows = len(df)
print(f"Removed {initial_rows - final_rows} rows with empty categories.")

Removed 3988 rows with empty categories.


In [8]:
print("Number of unique values per column:")
for column in df.columns:
    print(f"- {column}: {df[column].nunique()} unique values")

Number of unique values per column:
- categories: 21125 unique values
- employmentTypes: 8 unique values
- metadata_expiryDate: 453 unique values
- metadata_isPostedOnBehalf: 2 unique values
- metadata_jobPostId: 1044597 unique values
- metadata_newPostingDate: 431 unique values
- metadata_originalPostingDate: 603 unique values
- metadata_repostCount: 3 unique values
- metadata_totalNumberJobApplication: 376 unique values
- metadata_totalNumberOfView: 1552 unique values
- minimumYearsExperience: 50 unique values
- numberOfVacancies: 107 unique values
- occupationId: 0 unique values
- positionLevels: 9 unique values
- postedCompany_name: 53151 unique values
- salary_maximum: 2620 unique values
- salary_minimum: 2137 unique values
- salary_type: 1 unique values
- status_id: 1 unique values
- status_jobStatus: 3 unique values
- title: 377084 unique values
- average_salary: 4276 unique values
- category_name: 21125 unique values


In [9]:
# if company name contains 'kpmg', show list of such rows
df[df['postedCompany_name'].str.contains('uob', case=False, na=False)].sort_values('metadata_originalPostingDate', ascending=False)


,categories,employmentTypes,metadata_expiryDate,metadata_isPostedOnBehalf,metadata_jobPostId,metadata_newPostingDate,metadata_originalPostingDate,metadata_repostCount,metadata_totalNumberJobApplication,metadata_totalNumberOfView,...,positionLevels,postedCompany_name,salary_maximum,salary_minimum,salary_type,status_id,status_jobStatus,title,average_salary,category_name
1037650,"[{""id"":5,""category"":""Banking and Finance""}]",Full Time,2024-06-23,False,MCF-2024-0755539,2024-05-24,2024-05-24,0,1,32,...,Senior Executive,UOB KAY HIAN PRIVATE LIMITED,13000,8000,Monthly,0,Open,Senior Equity Analyst,10500.0,Banking and Finance
1032783,"[{""id"":5,""category"":""Banking and Finance""}]",Contract,2024-06-22,False,MCF-2024-0749486,2024-05-23,2024-05-23,0,1,11,...,Junior Executive,UOB KAY HIAN PRIVATE LIMITED,3500,2500,Monthly,0,Open,Client Accounts Officer - 1 year contract (Scr...,3000.0,Banking and Finance
1031421,"[{""id"":5,""category"":""Banking and Finance""}]",Permanent,2024-06-22,False,MCF-2024-0745599,2024-05-23,2024-05-23,0,1,8,...,Executive,UOB KAY HIAN PRIVATE LIMITED,5500,3500,Monthly,0,Open,"Director, Private Wealth Management",4500.0,Banking and Finance
1011338,"[{""id"":5,""category"":""Banking and Finance""}]",Full Time,2024-06-13,False,MCF-2024-0700071,2024-05-14,2024-05-14,0,1,15,...,Junior Executive,UOB KAY HIAN PRIVATE LIMITED,4000,2700,Monthly,0,Open,Associate - Corporate Action,3350.0,Banking and Finance
997691,"[{""id"":5,""category"":""Banking and Finance""}]",Contract,2024-06-08,False,MCF-2024-0676338,2024-05-09,2024-05-09,0,1,16,...,Junior Executive,UOB KAY HIAN PRIVATE LIMITED,4000,3000,Monthly,0,Open,Associate-Corporate Action (1 year contract),3500.0,Banking and Finance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25624,"[{""id"":5,""category"":""Banking and Finance""},{""i...",Permanent,2023-06-18,False,MCF-2023-0243564,2023-05-19,2023-03-28,1,33,425,...,Junior Executive,UOB KAY HIAN PRIVATE LIMITED,3500,2800,Monthly,0,Closed,Settlement Officer,3150.0,"Banking and Finance, Others"
25527,"[{""id"":1,""category"":""Accounting / Auditing / T...",Permanent,2023-04-27,False,MCF-2023-0243678,2023-03-28,2023-03-28,0,38,388,...,Junior Executive,UOB KAY HIAN PRIVATE LIMITED,4500,2500,Monthly,0,Closed,"Associate, Corporate Finance",3500.0,Accounting / Auditing / Taxation
25513,"[{""id"":5,""category"":""Banking and Finance""},{""i...",Permanent,2023-04-27,False,MCF-2023-0243691,2023-03-28,2023-03-28,0,45,354,...,Executive,UOB KAY HIAN PRIVATE LIMITED,4000,2500,Monthly,0,Closed,IT Security Analyst,3250.0,"Banking and Finance, Information Technology"
25504,"[{""id"":5,""category"":""Banking and Finance""}]",Permanent,2023-04-27,False,MCF-2023-0243685,2023-03-28,2023-03-28,0,15,168,...,Executive,UOB KAY HIAN PRIVATE LIMITED,4000,3000,Monthly,0,Closed,"Associate, Structured Finance",3500.0,Banking and Finance
